# A (very) simple TrueSkill model

The TrueSkill model (for details, see notebook 2b) is a probabilistic graphical model which is used in online gaming, in particular for XBox platform.

The TrueSkill model is a replacement for the Chess ELO skill estimate system, which converges faster, handles team play and with approximate inference using message passing, can be run online for about 100M players.  The objective of the system is to find players, who are well matched in skills so that they have fun playing.

TrueSkill models game outcomes based on **player skill** $s_i$, which affects **game performance** of an individual in a game. The performance in teams is estimates as the sum of performance of the team players. At this point, we diverge in our example and assume that the difference in performance predicts the difference in score.  The model is shown on the illustration below.

![Simplified TrueSkill schematic](images/simplified_trueskill.png "Simplified TrueSkill schematic")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import theano.tensor as tt
import theano

%matplotlib inline
sns.set_context('paper')
sns.set_style('darkgrid')

In [ ]:
score_differences_observed = [50, 40, 0, -10, 60, 35]

# setup model
with pm.Model() as simple_trueskill:
    
    mu0, sigma0 = 100, 10
    beta = 5
    gamma = 5
    
    s1 = pm.Normal('s1', mu=mu0, sd=sigma0)
    s2 = pm.Normal('s2', mu=mu0, sd=sigma0)
    s3 = pm.Normal('s3', mu=mu0, sd=sigma0)
    s4 = pm.Normal('s4', mu=mu0, sd=sigma0)
    
    p1 = pm.Normal('p1', mu=s1, sd=beta)
    p2 = pm.Normal('p2', mu=s2, sd=beta)
    p3 = pm.Normal('p3', mu=s3, sd=beta)
    p4 = pm.Normal('p4', mu=s4, sd=beta)
    
    t1 = pm.Deterministic('t1', p1+p2)
    t2 = pm.Deterministic('t2', p3+p4)
    
    d12 = pm.Normal('d12', mu=t1-t2, sd=gamma, observed = score_differences_observed)

In [ ]:
with simple_trueskill:
    point_estimate = pm.find_MAP(model=simple_trueskill)
    
point_estimate

In [ ]:
# fit model
with simple_trueskill:
    trace = pm.sample(draws=5000, tune=1000)

In [ ]:
_ = pm.traceplot(trace, varnames=['s1', 's2', 's3', 's4'])

### Exercises [easier]

1. How do you interpret the results of the model?
2. What if you already had prior information on one of the gamers, meaning their $\sigma_0$ would be smaller and she would have possibly a different $\mu_0$? What do you predict would happen if you modify the model that way?
3. After updating the distributions they are no longer Normally distributed.  What could you do to continue updating if you see 5 more games played?

### Exercise [harder]

What if you only observed wins/losses/draws in the game?

Here is a possible model, again inspired by TrueSkill but modified:
- if $t_i - t_j > \epsilon$, then $t_i$ wins with probability 90%, there is a draw with 9% probability and with 1% probability, team $i$ loses
- if $|t_i - t_j| <= \epsilon$, then $t_i$ and $t_j$ draw with 90% and win/lose situations (i.e. the complement) happen with 10% probability

Let us set $\epsilon = 10$.

As a hint: add a factor potential per game to the likelihood into the model below.  

In [ ]:
# games between players
game_outcomes_observed = [1, 1, 0, -1, 1, 1]

# setup model
with pm.Model() as reduced_trueskill:
    
    mu0, sigma0 = 100, 10
    beta = 5
    gamma = 5
    epsilon = 10
    
    s1 = pm.Normal('s1', mu=mu0, sd=sigma0)
    s2 = pm.Normal('s2', mu=mu0, sd=sigma0)
    
    p1 = pm.Normal('p1', mu=s1, sd=beta)
    p2 = pm.Normal('p2', mu=s2, sd=beta)
        
    game_potentials = []
    for i,y in enumerate(game_outcomes_observed):
        name = 'game_%d' % (i+1)
        if y==0:
            p = pm.Potential(name, pm.math.switch(pm.math.abs_(p1 - p2) < epsilon, np.log(0.9), np.log(0.1)))
            game_potentials.append(p)
        elif y==1:
            p = pm.Potential(name, pm.math.switch(p1 - p2 > epsilon, np.log(0.9), 
                                                  pm.math.switch(p2 - p1 > epsilon, np.log(0.01), np.log(0.09))))
            game_potentials.append(p)
        elif y==-1:
            p = pm.Potential(name, pm.math.switch(p2 - p1 > epsilon, np.log(0.9), 
                                                  pm.math.switch(p1 - p2 > epsilon, np.log(0.01), np.log(0.09))))
            game_potentials.append(p)
            

In [ ]:
game_potentials

In [ ]:
with reduced_trueskill:
    m = pm.find_MAP(method='powell')
print(m)

The MAP solver is not really finding a direction in which to maximize the MAP.

In [ ]:
with reduced_trueskill:
    tr = pm.sample(draws=20000, tune=1000, step=pm.Metropolis())

In [ ]:
_ = pm.traceplot(tr, varnames=['s1', 's2'])